In [7]:
import pandas as pd

read data:

In [54]:
df = pd.read_csv(r"C:\Users\Z E Y T O N\Desktop\Q-final-project\QBC-DA10-Group4-FinalProject\ML_Q1.csv", encoding="utf-8-sig")

print(df.shape)
df.head()

(30000, 11)


,Order ID,Product ID,Discount,Quantity,Sales,Sub-Category,Category,Shipping Cost,Markets,Segment,Profit
0,AE-2011-9160,OFF-FEL-10001405,0.7,2,82.674004,Storage,Office Supplies,5.690000,EMEA,Consumer,-157.085999
1,AE-2011-9160,TEC-EPS-10004171,0.7,6,78.407997,Machines,Technology,3.870000,EMEA,Consumer,-88.991997
2,AE-2013-1130,FUR-BUS-10003055,0.7,6,224.748001,Bookcases,Furniture,60.080002,EMEA,Consumer,-232.272003
3,AE-2013-1130,OFF-ACC-10004278,0.7,1,4.248000,Fasteners,Office Supplies,0.100000,EMEA,Consumer,-4.692000
4,AE-2014-2840,OFF-ROG-10001340,0.7,1,42.480000,Storage,Office Supplies,8.040000,EMEA,Consumer,-75.059998


In [55]:
df1 = df.copy()
df1.columns = [i.strip().replace(" ", "_") for i in df.columns]

print(df1.columns.tolist())
df1.head()

['Order_ID', 'Product_ID', 'Discount', 'Quantity', 'Sales', 'Sub-Category', 'Category', 'Shipping_Cost', 'Markets', 'Segment', 'Profit']


,Order_ID,Product_ID,Discount,Quantity,Sales,Sub-Category,Category,Shipping_Cost,Markets,Segment,Profit
0,AE-2011-9160,OFF-FEL-10001405,0.7,2,82.674004,Storage,Office Supplies,5.690000,EMEA,Consumer,-157.085999
1,AE-2011-9160,TEC-EPS-10004171,0.7,6,78.407997,Machines,Technology,3.870000,EMEA,Consumer,-88.991997
2,AE-2013-1130,FUR-BUS-10003055,0.7,6,224.748001,Bookcases,Furniture,60.080002,EMEA,Consumer,-232.272003
3,AE-2013-1130,OFF-ACC-10004278,0.7,1,4.248000,Fasteners,Office Supplies,0.100000,EMEA,Consumer,-4.692000
4,AE-2014-2840,OFF-ROG-10001340,0.7,1,42.480000,Storage,Office Supplies,8.040000,EMEA,Consumer,-75.059998


In [56]:
num_cols = ["Discount", "Quantity", "Sales", "Shipping_Cost", "Profit"]
obj_cols = df.select_dtypes(include="object").columns

print("NA counts in each column:\n", df.isna().sum())
print("\nduplicate rows:", df1.duplicated().sum())

df1[num_cols].describe()

NA counts in each column:
 Order ID         0
Product ID       0
Discount         0
Quantity         0
Sales            0
Sub-Category     0
Category         0
Shipping Cost    0
Markets          0
Segment          0
Profit           0
dtype: int64

duplicate rows: 0


,Discount,Quantity,Sales,Shipping_Cost,Profit
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.137641,3.480767,240.815577,25.627541,29.951294
std,0.207489,2.263154,482.209505,56.512899,175.900822
min,0.000000,1.000000,0.444000,0.002000,-6599.979980
25%,0.000000,2.000000,29.600000,2.509500,0.510000
50%,0.000000,3.000000,82.037998,7.590000,9.480000
75%,0.200000,5.000000,245.220001,23.590000,36.316350
max,0.800000,14.000000,17499.900391,933.570007,8399.980469


In [57]:
#  has_discount: 0/1
df1["has_discount"] = (df1["Discount"] > 0).astype(int)

#  unit_price: Sales per unit
df1["unit_price"] = df1["Sales"] / df1["Quantity"]

print(df1.head())

       Order_ID        Product_ID  Discount  Quantity       Sales  \
0  AE-2011-9160  OFF-FEL-10001405       0.7         2   82.674004   
1  AE-2011-9160  TEC-EPS-10004171       0.7         6   78.407997   
2  AE-2013-1130  FUR-BUS-10003055       0.7         6  224.748001   
3  AE-2013-1130  OFF-ACC-10004278       0.7         1    4.248000   
4  AE-2014-2840  OFF-ROG-10001340       0.7         1   42.480000   

  Sub-Category         Category  Shipping_Cost Markets   Segment      Profit  \
0      Storage  Office Supplies       5.690000    EMEA  Consumer -157.085999   
1     Machines       Technology       3.870000    EMEA  Consumer  -88.991997   
2    Bookcases        Furniture      60.080002    EMEA  Consumer -232.272003   
3    Fasteners  Office Supplies       0.100000    EMEA  Consumer   -4.692000   
4      Storage  Office Supplies       8.040000    EMEA  Consumer  -75.059998   

   has_discount  unit_price  
0             1   41.337002  
1             1   13.068000  
2             

In [58]:
# Extract order_year
df1["order_year"] = (
    df1["Order_ID"]
    .str.split("-", n=2, expand=True)[1]
    .str.extract(r"(\d{4})")[0]
    .astype(int)
)

print(df1[["Order_ID", "order_year"]].head())
print(df1["order_year"].value_counts().sort_index())

       Order_ID  order_year
0  AE-2011-9160        2011
1  AE-2011-9160        2011
2  AE-2013-1130        2013
3  AE-2013-1130        2013
4  AE-2014-2840        2014
order_year
2011    5618
2012    6753
2013    8041
2014    9588
Name: count, dtype: int64


In [59]:
# One-Hot encode categoricals and build X, y

numeric_feats = ["Discount", "Quantity", "Sales", "Shipping_Cost", "has_discount", "unit_price", "order_year"]
categorical_feats = ["Category", "Sub-Category", "Markets", "Segment"]

# 2) One-Hot (0/1) for categoricals
X_cat_oh = pd.get_dummies(df1[categorical_feats], drop_first=False, dtype=int)

# 3) Combine numeric + one-hot
X = pd.concat([df1[numeric_feats].reset_index(drop=True), X_cat_oh.reset_index(drop=True)], axis=1)

# 4) Target
y = df1["Profit"].copy()

print("X shape:", X.shape)
print("y shape:", y.shape)
X.head()

X shape: (30000, 36)
y shape: (30000,)


,Discount,Quantity,Sales,Shipping_Cost,has_discount,unit_price,order_year,Category_Furniture,Category_Office Supplies,Category_Technology,...,Sub-Category_Tables,Markets_APAC,Markets_Africa,Markets_Canada,Markets_EMEA,Markets_EU,Markets_LATAM,Markets_US,Segment_Consumer,Segment_Corporate
0,0.7,2,82.674004,5.690000,1,41.337002,2011,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,0.7,6,78.407997,3.870000,1,13.068000,2011,0,0,1,...,0,0,0,0,1,0,0,0,1,0
2,0.7,6,224.748001,60.080002,1,37.458000,2013,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0.7,1,4.248000,0.100000,1,4.248000,2013,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,0.7,1,42.480000,8.040000,1,42.480000,2014,0,1,0,...,0,0,0,0,1,0,0,0,1,0


In [60]:
from sklearn.model_selection import train_test_split

# split data to train/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

#split test into validation/test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(len(X_train), len(X_val), len(X_test))

21000 4500 4500


In [61]:
from sklearn.preprocessing import StandardScaler

num_cols = ["Discount","Quantity","Sales","Shipping_Cost","has_discount","unit_price","order_year"]

scaler = StandardScaler()
# fit on train only
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
# transform val/test with same scaler
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

In [64]:
import numpy as np

bounds = {}
for i in num_cols:
    q1 = X_train[i].quantile(0.25)
    q3 = X_train[i].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    bounds[i] = (low, high)
    # clip train
    X_train[i] = X_train[i].clip(lower=low, upper=high)

# apply same bounds to val/test
for i in num_cols:
    low, high = bounds[i]
    X_val[i]  = X_val[i].clip(lower=low, upper=high)
    X_test[i] = X_test[i].clip(lower=low, upper=high)

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# fit baseline model
linr = LinearRegression()
linr.fit(X_train, y_train)

# evaluate on TRAIN
train_preds = linr.predict(X_train)
train_mae  = mean_absolute_error(y_train, train_preds)
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
train_r2   = r2_score(y_train, train_preds)

# evaluate on VAL
val_preds = linr.predict(X_val)
val_mae  = mean_absolute_error(y_val, val_preds)
val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
val_r2   = r2_score(y_val, val_preds)

print(f"Train : MAE: {train_mae:.2f} , RMSE: {train_rmse:.2f} , R2: {train_r2:.3f}")
print(f"Val : MAE: {val_mae:.2f} , RMSE: {val_rmse:.2f} , R2: {val_r2:.3f}")

Train : MAE: 61.54 , RMSE: 169.83 , R2: 0.161
Val : MAE: 60.03 , RMSE: 138.59 , R2: 0.202


In [70]:
from sklearn.linear_model import Ridge

alphas = [0.01, 0.1, 1, 10, 100]
results = []

for a in alphas:
    # train ridge
    ridge = Ridge(alpha=a, random_state=42)
    ridge.fit(X_train, y_train)

    # evaluate
    tr_pred = ridge.predict(X_train)
    va_pred = ridge.predict(X_val)

    tr_mae  = mean_absolute_error(y_train, tr_pred)
    tr_rmse = np.sqrt(mean_squared_error(y_train, tr_pred))
    tr_r2   = r2_score(y_train, tr_pred)

    va_mae  = mean_absolute_error(y_val, va_pred)
    va_rmse = np.sqrt(mean_squared_error(y_val, va_pred))
    va_r2   = r2_score(y_val, va_pred)

    results.append((a, tr_mae, tr_rmse, tr_r2, va_mae, va_rmse, va_r2))


print("alpha | Train(MAE, RMSE, R2) ;    Val(MAE, RMSE, R2)")
for a, tr_mae, tr_rmse, tr_r2, va_mae, va_rmse, va_r2 in results:
    print(f"{a:<5} | {tr_mae:7.2f}, {tr_rmse:7.2f}, {tr_r2:5.3f} ; {va_mae:7.2f}, {va_rmse:7.2f}, {va_r2:5.3f}")

alpha | Train(MAE, RMSE, R2) ;    Val(MAE, RMSE, R2)
0.01  |   61.54,  169.83, 0.161 ;   60.02,  138.59, 0.202
0.1   |   61.54,  169.83, 0.161 ;   60.02,  138.59, 0.202
1     |   61.54,  169.83, 0.161 ;   60.02,  138.59, 0.202
10    |   61.50,  169.83, 0.161 ;   59.98,  138.59, 0.202
100   |   61.25,  169.89, 0.160 ;   59.67,  138.67, 0.201


In [72]:
from sklearn.linear_model import Lasso

alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
results = []

for a in alphas:
    # fit lasso
    lasso = Lasso(alpha=a, max_iter=10000, random_state=42)
    lasso.fit(X_train, y_train)

    # predictions
    tr_pred = lasso.predict(X_train)
    va_pred = lasso.predict(X_val)

    # metrics
    tr_mae  = mean_absolute_error(y_train, tr_pred)
    tr_rmse = np.sqrt(mean_squared_error(y_train, tr_pred))
    tr_r2   = r2_score(y_train, tr_pred)

    va_mae  = mean_absolute_error(y_val, va_pred)
    va_rmse = np.sqrt(mean_squared_error(y_val, va_pred))
    va_r2   = r2_score(y_val, va_pred)

    # sparsity: number of non-zero coefficients
    nnz = np.sum(lasso.coef_ != 0)

    results.append((a, nnz, tr_mae, tr_rmse, tr_r2, va_mae, va_rmse, va_r2))

print("alpha | non-zero | Train(MAE, RMSE, R2) ; Val(MAE, RMSE, R2)")
for a, nnz, tr_mae, tr_rmse, tr_r2, va_mae, va_rmse, va_r2 in results:
    print(f"{a:<6} | {nnz:8d} | {tr_mae:7.2f}, {tr_rmse:7.2f}, {tr_r2:5.3f} ; {va_mae:7.2f}, {va_rmse:7.2f}, {va_r2:5.3f}")

C:\anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.115e+06, tolerance: 7.216e+04
  model = cd_fast.enet_coordinate_descent(


alpha | non-zero | Train(MAE, RMSE, R2) ; Val(MAE, RMSE, R2)
0.0001 |       33 |   61.54,  169.83, 0.161 ;   60.02,  138.59, 0.202
0.001  |       31 |   61.54,  169.83, 0.161 ;   60.02,  138.59, 0.202
0.01   |       32 |   61.52,  169.83, 0.161 ;   59.99,  138.59, 0.202
0.1    |       28 |   61.36,  169.85, 0.160 ;   59.77,  138.57, 0.202
1      |       13 |   61.28,  170.55, 0.154 ;   59.33,  139.13, 0.196
10     |        3 |   60.64,  173.51, 0.124 ;   58.15,  141.78, 0.165


In [76]:
from sklearn.ensemble import RandomForestRegressor

# simple grid of hyperparameters
param_grid = [
    {"n_estimators": 100, "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 10,   "min_samples_split": 2, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 10,   "min_samples_split": 5, "min_samples_leaf": 2},
    {"n_estimators": 500, "max_depth": 15,   "min_samples_split": 5, "min_samples_leaf": 2},
]

train_metrics = []
val_metrics = []

# train and evaluate
for p in param_grid:
    rf = RandomForestRegressor(
        n_estimators=p["n_estimators"],
        max_depth=p["max_depth"],
        min_samples_split=p["min_samples_split"],
        min_samples_leaf=p["min_samples_leaf"],
        n_jobs=-1,
        random_state=42,
    )
    rf.fit(X_train, y_train)

    # train metrics
    tr_pred = rf.predict(X_train)
    tr_mae  = mean_absolute_error(y_train, tr_pred)
    tr_rmse = np.sqrt(mean_squared_error(y_train, tr_pred))
    tr_r2   = r2_score(y_train, tr_pred)
    train_metrics.append((p, tr_mae, tr_rmse, tr_r2))

    # val metrics
    va_pred = rf.predict(X_val)
    va_mae  = mean_absolute_error(y_val, va_pred)
    va_rmse = np.sqrt(mean_squared_error(y_val, va_pred))
    va_r2   = r2_score(y_val, va_pred)
    val_metrics.append((p, va_mae, va_rmse, va_r2))

# print train results first
print("TRAIN RESULTS:")
for p, mae, rmse, r2 in train_metrics:
    print(f"{p} -> MAE: {mae:.2f} | RMSE: {rmse:.2f} | R2: {r2:.3f}")

# then print val results in the same order
print("\nVALIDATION RESULTS:")
for p, mae, rmse, r2 in val_metrics:
    print(f"{p} -> MAE: {mae:.2f} | RMSE: {rmse:.2f} | R2: {r2:.3f}")

TRAIN RESULTS:
{'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1} -> MAE: 14.66 | RMSE: 53.47 | R2: 0.917
{'n_estimators': 300, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1} -> MAE: 14.65 | RMSE: 54.40 | R2: 0.914
{'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1} -> MAE: 28.84 | RMSE: 82.44 | R2: 0.802
{'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2} -> MAE: 30.12 | RMSE: 99.25 | R2: 0.713
{'n_estimators': 500, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 2} -> MAE: 23.72 | RMSE: 89.30 | R2: 0.768

VALIDATION RESULTS:
{'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1} -> MAE: 36.93 | RMSE: 120.66 | R2: 0.395
{'n_estimators': 300, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1} -> MAE: 36.66 | RMSE: 117.31 | R2: 0.428
{'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 2, 'min_samp

In [78]:
from xgboost import XGBRegressor

#base model
xgb = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# fit
xgb.fit(X_train, y_train)

# evaluate on train
train_pred = xgb.predict(X_train)
train_mae  = mean_absolute_error(y_train, train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
train_r2   = r2_score(y_train, train_pred)

# evaluate on val
val_pred = xgb.predict(X_val)
val_mae  = mean_absolute_error(y_val, val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
val_r2   = r2_score(y_val, val_pred)

print(f"Train -> MAE: {train_mae:.2f} | RMSE: {train_rmse:.2f} | R2: {train_r2:.3f}")
print(f"Val   -> MAE: {val_mae:.2f} | RMSE: {val_rmse:.2f} | R2: {val_r2:.3f}")

Train -> MAE: 34.15 | RMSE: 99.83 | R2: 0.710
Val   -> MAE: 37.70 | RMSE: 108.25 | R2: 0.513
